# Oracle Database Integration with Docker and Vector Storage Visualised in Voila, (Simple)

This notebook demonstrates:
1. Setting up Oracle Database using Docker
2. Connecting to Oracle DB from Python
3. Vector storage preparation
4. Creating an interactive web interface with Voilà

## Prerequisites
- Docker installed
- Python 3.8+
- uv.docker access

## 1. Environment Setup

First, let's install required packages:

In [2]:
# !pip install oracledb ipywidgets voila docker numpy pandas plotly matplotlib seaborn bokeh

In [3]:
import oracledb
import docker
import ipywidgets as widgets
from IPython.display import display, HTML
import time
import os
import random
from datetime import datetime, timedelta

# Data analysis and visualization
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()  # Enable Bokeh output in notebook

Loading BokehJS ...

### Setting up Oracle Database Container

In [4]:
def setup_oracle_container():
    client = docker.from_env()
    
    # Oracle DB container configuration
    container_name = 'oracle-xe'
    image_name = 'container-registry.oracle.com/database/express:21.3.0-xe'
    
    # Environment variables
    environment = {
        'ORACLE_PWD': 'your_secure_password',  # Change this!
        'ORACLE_CHARACTERSET': 'AL32UTF8'
    }
    
    # Volume configuration
    volumes = {
        'oracle_data': {'bind': '/opt/oracle/oradata', 'mode': 'rw'}
    }
    
    # Ports configuration
    ports = {
        '1521/tcp': 1521  # Oracle listener port
    }
    
    try:
        # Check if container already exists
        try:
            existing_container = client.containers.get(container_name)
            
            # If container exists but is not running, remove it
            if existing_container.status != 'running':
                print(f"Removing stopped container {container_name}...")
                existing_container.remove(force=True)
            else:
                print(f"Container {container_name} is already running")
                return existing_container
                
        except docker.errors.NotFound:
            # Container doesn't exist, proceed with creation
            pass
        
        # Pull the image
        print("Pulling Oracle Database image...")
        try:
            client.images.pull(image_name)
        except docker.errors.APIError as e:
            if 'authentication required' in str(e).lower():
                print("Error: Authentication required for Oracle Container Registry.")
                print("Please visit https://container-registry.oracle.com and create an account")
                print("Then run: docker login container-registry.oracle.com")
                return None
            raise
        
        # Create and start the container
        container = client.containers.run(
            image_name,
            name=container_name,
            environment=environment,
            volumes=volumes,
            ports=ports,
            detach=True
        )
        
        print("Container started. Waiting for database initialization...")
        
        # Wait for database to initialize (check logs for readiness)
        max_wait = 300  # 5 minutes timeout
        start_time = time.time()
        while time.time() - start_time < max_wait:
            logs = container.logs().decode('utf-8')
            if "DATABASE IS READY TO USE!" in logs:
                print("Database is ready!")
                return container
            time.sleep(10)
            
        print("Warning: Database initialization timeout reached")
        return container
    
    except Exception as e:
        print(f"Error: {str(e)}")
        print("\nTroubleshooting steps:")
        print("1. Check if Docker is running")
        print("2. Verify Oracle Container Registry credentials")
        print("3. Ensure ports are not in use (netstat -an | grep 1521)")
        print("4. Check available disk space (df -h)")
        return None

### Initialize Oracle Container

In [5]:
container = setup_oracle_container()

Container oracle-xe is already running


## 2. Database Connection Setup

In [6]:
def create_connection():
    try:
        connection = oracledb.connect(
            user="system",
            password="your_secure_password",  # Same as ORACLE_PWD
            dsn="localhost:1521/XEPDB1"
        )
        return connection
    except Exception as e:
        print(f"Connection error: {str(e)}")
        return None

# Test connection
def test_connection():
    connection = create_connection()
    if connection:
        with connection.cursor() as cursor:
            cursor.execute("SELECT 1 FROM DUAL")
            result = cursor.fetchone()
            print("Connection successful!", result)
        connection.close()
    else:
        print("Connection failed!")

In [7]:
test_connection()

Connection successful! (1,)


## 3. Vector Storage Preparation

Create a table for storing vector embeddings using BLOB type:

In [8]:
def setup_vector_storage():
    connection = create_connection()
    if connection:
        with connection.cursor() as cursor:
            # Create table for vector storage
            cursor.execute("""
                CREATE TABLE vector_embeddings (
                    id NUMBER GENERATED ALWAYS AS IDENTITY,
                    text_data VARCHAR2(4000),
                    embedding BLOB,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    PRIMARY KEY (id)
                )
            """)
        connection.commit()
        connection.close()
        print("Vector storage table created successfully!")

## Data Generation and Table Creation Utilities

This section provides utilities for:

1. **Generating Dummy Data**
   - Configurable number of records
   - Support for various data types:
     - Strings (categorical or free text)
     - Integers
     - Floats
     - Dates
     - Vector embeddings
   - Control over NULL values
   - Custom ranges and choices

2. **Table Creation**
   - Automatic schema generation from DataFrame
   - Proper handling of vector data as BLOB
   - Support for various Oracle data types
   - Automatic ID generation
   - Timestamp tracking

In [9]:
def generate_dummy_data(config=None):
    """
    Generate dummy data based on configuration
    """
    if config is None:
        # Default configuration
        config = {
            'num_records': 100,
            'fields': {
                'text_content': {
                    'type': 'string',
                    'choices': ['Sample text A', 'Sample text B', 'Sample text C'],
                    'null_prob': 0.1
                },
                'category': {
                    'type': 'string',
                    'choices': ['cat1', 'cat2', 'cat3'],
                    'null_prob': 0.05
                },
                'value': {
                    'type': 'float',
                    'min': 0,
                    'max': 100,
                    'null_prob': 0.1
                },
                'count': {
                    'type': 'int',
                    'min': 1,
                    'max': 1000,
                    'null_prob': 0
                },
                'embedding': {
                    'type': 'vector',
                    'vector_dim': 128,
                    'null_prob': 0
                },
                'created_date': {
                    'type': 'date',
                    'min': '2025-01-01',
                    'max': '2025-12-31',
                    'null_prob': 0
                }
            }
        }
    
    data = []
    for _ in range(config['num_records']):
        record = {}
        for field_name, field_config in config['fields'].items():
            # Check for NULL value
            if random.random() < field_config.get('null_prob', 0):
                record[field_name] = None
                continue
                
            field_type = field_config['type']
            if field_type == 'string':
                choices = field_config.get('choices', ['dummy_value'])
                record[field_name] = random.choice(choices)
            
            elif field_type == 'int':
                min_val = field_config.get('min', 0)
                max_val = field_config.get('max', 100)
                record[field_name] = random.randint(min_val, max_val)
            
            elif field_type == 'float':
                min_val = field_config.get('min', 0)
                max_val = field_config.get('max', 100)
                record[field_name] = random.uniform(min_val, max_val)
            
            elif field_type == 'date':
                min_date = datetime.strptime(field_config.get('min', '2025-01-01'), '%Y-%m-%d')
                max_date = datetime.strptime(field_config.get('max', '2025-12-31'), '%Y-%m-%d')
                days_between = (max_date - min_date).days
                random_days = random.randint(0, days_between)
                record[field_name] = min_date + timedelta(days=random_days)
            
            elif field_type == 'vector':
                dim = field_config.get('vector_dim', 128)
                vector = np.random.randn(dim)
                # Normalize the vector
                record[field_name] = vector / np.linalg.norm(vector)
        
        data.append(record)
    
    return pd.DataFrame(data)

def generate_create_table_sql(df, table_name, vector_cols=None):
    """
    Generate CREATE TABLE SQL statement based on DataFrame schema
    """
    type_mapping = {
        'object': 'VARCHAR2(4000)',
        'int64': 'NUMBER',
        'float64': 'FLOAT',
        'datetime64[ns]': 'TIMESTAMP',
        'bool': 'NUMBER(1)',
    }
    
    if vector_cols is None:
        vector_cols = []
    
    columns = []
    for col in df.columns:
        if col in vector_cols:
            col_type = 'BLOB'
        else:
            dtype = str(df[col].dtype)
            col_type = type_mapping.get(dtype, 'VARCHAR2(4000)')
        
        # Add NOT NULL if the column has no null values
        null_constraint = 'NOT NULL' if not df[col].isnull().any() else ''
        columns.append(f"{col} {col_type} {null_constraint}")
    
    create_sql = f"""
    CREATE TABLE {table_name} (
        id NUMBER GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
        {',\n        '.join(columns)},
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    return create_sql

def create_table_from_data(df, table_name, vector_cols=None, connection=None):
    """
    Create table and insert data from DataFrame
    """
    if connection is None:
        connection = create_connection()
    
    if connection:
        try:
            cursor = connection.cursor()
            
            # Create table
            create_sql = generate_create_table_sql(df, table_name, vector_cols)
            cursor.execute(create_sql)
            
            # Prepare insert statement
            columns = df.columns
            placeholders = ','.join([':' + str(i+1) for i in range(len(columns))])
            insert_sql = f"INSERT INTO {table_name} ({','.join(columns)}) VALUES ({placeholders})"
            
            # Insert data
            for _, row in df.iterrows():
                values = []
                for col in columns:
                    if col in vector_cols:
                        # Convert numpy array to bytes for BLOB storage
                        values.append(row[col].tobytes())
                    else:
                        values.append(row[col])
                cursor.execute(insert_sql, values)
            
            connection.commit()
            print(f"Successfully created table {table_name} and inserted {len(df)} records")
            
        except Exception as e:
            print(f"Error: {str(e)}")
            connection.rollback()
        finally:
            connection.close()

In [10]:
def get_table_schema(table_name, connection=None):
    """
    Get detailed schema information for a table
    """
    if connection is None:
        connection = create_connection()
    
    if connection:
        try:
            # Get column information
            columns_query = """
            SELECT column_name, data_type, data_length, nullable, data_precision, data_scale
            FROM user_tab_columns
            WHERE table_name = :1
            ORDER BY column_id
            """
            
            # Get constraint information
            constraints_query = """
            SELECT constraint_name, constraint_type, search_condition, r_owner, r_constraint_name
            FROM user_constraints
            WHERE table_name = :1
            """
            
            # Get index information
            indexes_query = """
            SELECT index_name, uniqueness, index_type
            FROM user_indexes
            WHERE table_name = :1
            """
            
            with connection.cursor() as cursor:
                # Get columns
                cursor.execute(columns_query, [table_name.upper()])
                columns = pd.DataFrame(cursor.fetchall(), 
                                     columns=['Column', 'Type', 'Length', 'Nullable', 'Precision', 'Scale'])
                
                # Get constraints
                cursor.execute(constraints_query, [table_name.upper()])
                constraints = pd.DataFrame(cursor.fetchall(),
                                        columns=['Name', 'Type', 'Condition', 'Ref_Owner', 'Ref_Constraint'])
                
                # Get indexes
                cursor.execute(indexes_query, [table_name.upper()])
                indexes = pd.DataFrame(cursor.fetchall(),
                                     columns=['Name', 'Uniqueness', 'Type'])
                
                return {
                    'columns': columns,
                    'constraints': constraints,
                    'indexes': indexes
                }
        
        except Exception as e:
            print(f"Error getting schema: {str(e)}")
            return None
        finally:
            connection.close()
    return None

def create_standard_tables():
    """
    Create a set of standardized tables for common use cases
    """
    # Document Store table
    doc_store_config = {
        'num_records': 0,  # No initial records
        'fields': {
            'title': {
                'type': 'string',
                'null_prob': 0.0
            },
            'content': {
                'type': 'string',
                'null_prob': 0.0
            },
            'doc_type': {
                'type': 'string',
                'choices': ['article', 'report', 'paper', 'note'],
                'null_prob': 0.0
            },
            'embedding': {
                'type': 'vector',
                'vector_dim': 384,  # Standard BERT embedding size
                'null_prob': 0.0
            },
            'metadata': {
                'type': 'string',  # JSON string
                'null_prob': 0.1
            }
        }
    }
    
    # Time Series table
    timeseries_config = {
        'num_records': 0,
        'fields': {
            'timestamp': {
                'type': 'date',
                'null_prob': 0.0
            },
            'metric_name': {
                'type': 'string',
                'null_prob': 0.0
            },
            'value': {
                'type': 'float',
                'null_prob': 0.0
            },
            'dimensions': {
                'type': 'string',  # JSON string for dimensions
                'null_prob': 0.1
            }
        }
    }
    
    # Entity table
    entity_config = {
        'num_records': 0,
        'fields': {
            'entity_type': {
                'type': 'string',
                'choices': ['person', 'organization', 'location'],
                'null_prob': 0.0
            },
            'name': {
                'type': 'string',
                'null_prob': 0.0
            },
            'attributes': {
                'type': 'string',  # JSON string
                'null_prob': 0.1
            },
            'embedding': {
                'type': 'vector',
                'vector_dim': 256,
                'null_prob': 0.0
            }
        }
    }
    
    # Create empty DataFrames and tables
    tables = {
        'document_store': (generate_dummy_data(doc_store_config), ['embedding']),
        'time_series': (generate_dummy_data(timeseries_config), []),
        'entities': (generate_dummy_data(entity_config), ['embedding'])
    }
    
    for table_name, (df, vector_cols) in tables.items():
        try:
            create_table_from_data(df, table_name, vector_cols)
            print(f"Created standard table: {table_name}")
            
            # Display schema
            schema = get_table_schema(table_name)
            if schema:
                print(f"\nSchema for {table_name}:")
                display(schema['columns'])
                if not schema['constraints'].empty:
                    print("\nConstraints:")
                    display(schema['constraints'])
                if not schema['indexes'].empty:
                    print("\nIndexes:")
                    display(schema['indexes'])
        except Exception as e:
            print(f"Error creating {table_name}: {str(e)}")

## 4. Interactive Web Interface with Voilà

In [11]:
# Basic interface for running SQL queries and visualizing results

def create_interface():
    # Create widgets
    query_input = widgets.Text(
        value='',
        placeholder='Enter your SQL query',
        description='Query:',
        disabled=False
    )
    
    output = widgets.Output()
    
    def on_button_clicked(b):
        with output:
            output.clear_output()
            connection = create_connection()
            if connection:
                try:
                    with connection.cursor() as cursor:
                        cursor.execute(query_input.value)
                        results = cursor.fetchall()
                        for row in results:
                            print(row)
                except Exception as e:
                    print(f"Error: {str(e)}")
                finally:
                    connection.close()
    
    button = widgets.Button(description="Execute Query")
    button.on_click(on_button_clicked)
    
    # Display widgets
    display(widgets.VBox([query_input, button, output]))

# Create the interface
create_interface()

In [12]:
def test_visualization():
    connection = create_connection()
    if connection:
        try:
            # Create sample data
            with connection.cursor() as cursor:
                cursor.execute("""
                    SELECT table_name, num_rows, avg_row_len 
                    FROM user_tables 
                    WHERE num_rows > 0
                """)
                df = pd.DataFrame(cursor.fetchall(), 
                                columns=['table_name', 'num_rows', 'avg_row_len'])
                
                # Create visualization
                fig = px.bar(df, x='table_name', y='num_rows',
                           title='Table Row Counts')
                fig.show()
                
                return df
        except Exception as e:
            print(f"Error: {str(e)}")
        finally:
            connection.close()
    return None

# Test the visualization
test_visualization()

,table_name,num_rows,avg_row_len
0,MVIEW$_ADV_PARAMETERS,40,30
1,AQ$_QUEUES,24,88
2,REPL_VALID_COMPAT,1,3
3,REPL_SUPPORT_MATRIX,82,39
4,AQ$_QUEUE_TABLES,12,81
5,AQ$_INTERNET_AGENTS,6,18
6,AQ$_INTERNET_AGENT_PRIVS,4,23
7,HELP,978,40
8,REDO_DB,1,90
9,LOGSTDBY$SKIP_SUPPORT,676,34


In [13]:
def create_interface():
    # Tab layout
    tab = widgets.Tab()
    
    # Query Tab
    query_input = widgets.Text(
        value='',
        placeholder='Enter your SQL query',
        description='Query:',
        disabled=False,
        layout=widgets.Layout(width='80%')
    )
    
    # Dropdown for common queries
    common_queries = widgets.Dropdown(
        options=[
            ('Show all tables', 'SELECT table_name FROM user_tables'),
            ('Vector embeddings data', 'SELECT * FROM vector_embeddings'),
            ('Table statistics', 'SELECT table_name, num_rows, avg_row_len FROM user_tables'),
            ('Database version', 'SELECT * FROM v$version')
        ],
        description='Quick Queries:',
        layout=widgets.Layout(width='80%')
    )
    
    output = widgets.Output()
    viz_output = widgets.Output()
    
    # Data Generation Tab
    num_records = widgets.IntSlider(
        value=100,
        min=10,
        max=1000,
        step=10,
        description='Records:',
        layout=widgets.Layout(width='50%')
    )
    
    table_name = widgets.Text(
        value='generated_data',
        placeholder='Enter table name',
        description='Table:',
        layout=widgets.Layout(width='50%')
    )
    
    vector_dim = widgets.IntSlider(
        value=128,
        min=32,
        max=512,
        step=32,
        description='Vector Dim:',
        layout=widgets.Layout(width='50%')
    )
    
    gen_output = widgets.Output()
    
    def generate_sample_data(b):
        with gen_output:
            gen_output.clear_output()
            
            config = {
                'num_records': num_records.value,
                'fields': {
                    'title': {
                        'type': 'string',
                        'choices': ['Research Paper', 'Article', 'Blog Post', 'News'],
                        'null_prob': 0.0
                    },
                    'content_length': {
                        'type': 'int',
                        'min': 100,
                        'max': 5000,
                        'null_prob': 0.0
                    },
                    'relevance_score': {
                        'type': 'float',
                        'min': 0.0,
                        'max': 1.0,
                        'null_prob': 0.1
                    },
                    'embedding': {
                        'type': 'vector',
                        'vector_dim': vector_dim.value,
                        'null_prob': 0.0
                    },
                    'publish_date': {
                        'type': 'date',
                        'min': '2025-01-01',
                        'max': '2025-12-31',
                        'null_prob': 0.0
                    }
                }
            }
            
            df = generate_dummy_data(config)
            print(f"Generated {len(df)} records")
            create_table_from_data(df, table_name.value, vector_cols=['embedding'])
            
            # Display sample and visualization
            display(df.head())
            
            fig = px.scatter(df,
                           x='publish_date',
                           y='content_length',
                           color='relevance_score',
                           title='Generated Data Overview')
            fig.show()
    
    generate_button = widgets.Button(description="Generate Data")
    generate_button.on_click(generate_sample_data)
    
    def query_to_df(query):
        connection = create_connection()
        if connection:
            try:
                return pd.read_sql(query, connection)
            except Exception as e:
                print(f"Error: {str(e)}")
                return None
            finally:
                connection.close()
        return None

    def create_visualizations(df):
        if df is None or df.empty:
            return
        
        with viz_output:
            viz_output.clear_output(wait=True)
            
            # Create basic statistics
            display(widgets.HTML("<h3>Data Statistics</h3>"))
            display(df.describe())
            
            # For numerical columns, create a box plot
            num_cols = df.select_dtypes(include=['int64', 'float64']).columns
            if len(num_cols) > 0:
                fig = px.box(df, y=num_cols)
                fig.update_layout(title="Numerical Columns Distribution")
                fig.show()
            
            # For categorical columns, create a bar plot
            cat_cols = df.select_dtypes(include=['object']).columns
            if len(cat_cols) > 0:
                for col in cat_cols[:3]:  # Limit to first 3 categorical columns
                    value_counts = df[col].value_counts()
                    fig = px.bar(x=value_counts.index, y=value_counts.values,
                               title=f"Distribution of {col}")
                    fig.show()

    def on_query_select(change):
        if change['type'] == 'change' and change['name'] == 'value':
            query_input.value = change['new']

    def on_button_clicked(b):
        with output:
            output.clear_output(wait=True)
            df = query_to_df(query_input.value)
            if df is not None:
                display(widgets.HTML("<h3>Query Results</h3>"))
                display(df)
                create_visualizations(df)

    common_queries.observe(on_query_select, names='value')
    button = widgets.Button(description="Execute Query")
    button.on_click(on_button_clicked)
    
    # Create tab contents
    query_tab = widgets.VBox([
        widgets.HTML("<h2>Database Query Interface</h2>"),
        common_queries,
        query_input,
        button,
        output,
        viz_output
    ])
    
    # Create data generation tab
    gen_tab = widgets.VBox([
        widgets.HTML("<h2>Generate Sample Data</h2>"),
        widgets.HBox([num_records, table_name]),
        vector_dim,
        generate_button,
        gen_output
    ])
    
    # Create info tab
    info_tab = widgets.VBox([
        widgets.HTML("""
        <h2>Information</h2>
        <h3>Available Tables:</h3>
        <ul>
            <li>vector_embeddings - Stores vector data and embeddings</li>
            <li>user_tables - System table with metadata about all tables</li>
            <li>v$version - Database version information</li>
        </ul>
        <h3>Tips:</h3>
        <ul>
            <li>Use the Quick Queries dropdown for common operations</li>
            <li>Results will be displayed as interactive dataframes</li>
            <li>Automatic visualizations are generated for numerical and categorical data</li>
            <li>Use the Generate Sample Data tab to create test data</li>
        </ul>
        """)
    ])
    
    # Set up tabs
    tab.children = [query_tab, gen_tab, info_tab]
    tab.set_title(0, 'Query & Visualize')
    tab.set_title(1, 'Generate Data')
    tab.set_title(2, 'Information')
    
    display(tab)

In [14]:
# create_interface()

## Running as a Web Application

To run this notebook as a web application:

1. Save this notebook
2. Open terminal
3. Run: `voila oracle_vector_demo.ipynb --port 8866`
4. Open browser at http://localhost:8866

### Notes:
- Ensure all credentials are properly secured in production
- Consider adding authentication for the web interface
- Add error handling and input validation for production use

## 5. Schema Management and Standard Tables

This section provides utilities for:
1. Viewing table schemas
2. Creating standardized tables for common use cases
3. Managing table metadata

In [15]:
def get_table_schema(table_name, connection=None):
    """
    Get detailed schema information for a table
    """
    if connection is None:
        connection = create_connection()
    
    if connection:
        try:
            # Get column information
            columns_query = """
            SELECT column_name, data_type, data_length, nullable, data_precision, data_scale
            FROM user_tab_columns
            WHERE table_name = :1
            ORDER BY column_id
            """
            
            # Get constraint information
            constraints_query = """
            SELECT constraint_name, constraint_type, search_condition, r_owner, r_constraint_name
            FROM user_constraints
            WHERE table_name = :1
            """
            
            # Get index information
            indexes_query = """
            SELECT index_name, uniqueness, index_type
            FROM user_indexes
            WHERE table_name = :1
            """
            
            with connection.cursor() as cursor:
                # Get columns
                cursor.execute(columns_query, [table_name.upper()])
                columns = pd.DataFrame(cursor.fetchall(), 
                                     columns=['Column', 'Type', 'Length', 'Nullable', 'Precision', 'Scale'])
                
                # Get constraints
                cursor.execute(constraints_query, [table_name.upper()])
                constraints = pd.DataFrame(cursor.fetchall(),
                                        columns=['Name', 'Type', 'Condition', 'Ref_Owner', 'Ref_Constraint'])
                
                # Get indexes
                cursor.execute(indexes_query, [table_name.upper()])
                indexes = pd.DataFrame(cursor.fetchall(),
                                     columns=['Name', 'Uniqueness', 'Type'])
                
                return {
                    'columns': columns,
                    'constraints': constraints,
                    'indexes': indexes
                }
        
        except Exception as e:
            print(f"Error getting schema: {str(e)}")
            return None
        finally:
            connection.close()
    return None

# Create a schema viewer interface
def create_schema_viewer():
    schema_tab = widgets.Tab()
    
    # Table selector
    table_select = widgets.Dropdown(
        options=[],
        description='Table:',
        layout=widgets.Layout(width='50%')
    )
    
    schema_output = widgets.Output()
    
    def update_table_list():
        connection = create_connection()
        if connection:
            try:
                with connection.cursor() as cursor:
                    cursor.execute("SELECT table_name FROM user_tables ORDER BY table_name")
                    tables = [row[0] for row in cursor.fetchall()]
                    table_select.options = tables
            finally:
                connection.close()
    
    def view_schema(change):
        if change['type'] == 'change' and change['name'] == 'value':
            with schema_output:
                schema_output.clear_output()
                schema = get_table_schema(change['new'])
                if schema:
                    display(widgets.HTML(f"<h3>Schema for {change['new']}</h3>"))
                    display(widgets.HTML("<h4>Columns:</h4>"))
                    display(schema['columns'])
                    if not schema['constraints'].empty:
                        display(widgets.HTML("<h4>Constraints:</h4>"))
                        display(schema['constraints'])
                    if not schema['indexes'].empty:
                        display(widgets.HTML("<h4>Indexes:</h4>"))
                        display(schema['indexes'])
    
    table_select.observe(view_schema, names='value')
    
    # Initialize table list
    update_table_list()
    
    # Layout
    return widgets.VBox([
        widgets.HTML("<h2>Table Schema Viewer</h2>"),
        table_select,
        schema_output
    ])

# Create and display the schema viewer
schema_viewer = create_schema_viewer()
display(schema_viewer)

In [16]:
# def create_standard_tables():
#     """
#     Create a set of standardized tables for common use cases
#     """
#     # Document Store table
#     doc_store_config = {
#         'num_records': 0,  # No initial records
#         'fields': {
#             'title': {
#                 'type': 'string',
#                 'null_prob': 0.0
#             },
#             'content': {
#                 'type': 'string',
#                 'null_prob': 0.0
#             },
#             'doc_type': {
#                 'type': 'string',
#                 'choices': ['article', 'report', 'paper', 'note'],
#                 'null_prob': 0.0
#             },
#             'embedding': {
#                 'type': 'vector',
#                 'vector_dim': 384,  # Standard BERT embedding size
#                 'null_prob': 0.0
#             },
#             'metadata': {
#                 'type': 'string',  # JSON string
#                 'null_prob': 0.1
#             }
#         }
#     }
    
#     # Time Series table
#     timeseries_config = {
#         'num_records': 0,
#         'fields': {
#             'timestamp': {
#                 'type': 'date',
#                 'null_prob': 0.0
#             },
#             'metric_name': {
#                 'type': 'string',
#                 'null_prob': 0.0
#             },
#             'value': {
#                 'type': 'float',
#                 'null_prob': 0.0
#             },
#             'dimensions': {
#                 'type': 'string',  # JSON string for dimensions
#                 'null_prob': 0.1
#             }
#         }
#     }
    
#     # Entity table
#     entity_config = {
#         'num_records': 0,
#         'fields': {
#             'entity_type': {
#                 'type': 'string',
#                 'choices': ['person', 'organization', 'location'],
#                 'null_prob': 0.0
#             },
#             'name': {
#                 'type': 'string',
#                 'null_prob': 0.0
#             },
#             'attributes': {
#                 'type': 'string',  # JSON string
#                 'null_prob': 0.1
#             },
#             'embedding': {
#                 'type': 'vector',
#                 'vector_dim': 256,
#                 'null_prob': 0.0
#             }
#         }
#     }
    
#     # Create empty DataFrames and tables
#     tables = {
#         'document_store': (generate_dummy_data(doc_store_config), ['embedding']),
#         'time_series': (generate_dummy_data(timeseries_config), []),
#         'entities': (generate_dummy_data(entity_config), ['embedding'])
#     }
    
#     for table_name, (df, vector_cols) in tables.items():
#         try:
#             create_table_from_data(df, table_name, vector_cols)
#             print(f"Created standard table: {table_name}")
            
#             # Display schema
#             schema = get_table_schema(table_name)
#             if schema:
#                 print(f"\nSchema for {table_name}:")
#                 display(schema['columns'])
#                 if not schema['constraints'].empty:
#                     print("\nConstraints:")
#                     display(schema['constraints'])
#                 if not schema['indexes'].empty:
#                     print("\nIndexes:")
#                     display(schema['indexes'])
#         except Exception as e:
#             print(f"Error creating {table_name}: {str(e)}")

# Create standard tables interface
def create_standard_tables_interface():
    output = widgets.Output()
    
    create_button = widgets.Button(
        description='Create Standard Tables',
        tooltip='Creates a set of predefined tables for common use cases'
    )
    
    def on_create_clicked(b):
        with output:
            output.clear_output()
            create_standard_tables()
    
    create_button.on_click(on_create_clicked)
    
    return widgets.VBox([
        widgets.HTML("""
        <h2>Standard Tables</h2>
        <p>Click the button below to create a set of standardized tables:</p>
        <ul>
            <li><strong>document_store</strong>: For storing documents with vector embeddings</li>
            <li><strong>time_series</strong>: For time-series data with dimensions</li>
            <li><strong>entities</strong>: For storing entity data with embeddings</li>
        </ul>
        """),
        create_button,
        output
    ])

# # Create and display the standard tables interface
# standard_tables = create_standard_tables_interface()
# display(standard_tables)

In [17]:
def create_interface():
    # Main tab layout
    main_tab = widgets.Tab()
    
    # Create all sub-components
    query_tab = create_query_tab()
    gen_tab = create_data_generation_tab()
    info_tab = create_info_tab()
    schema_tab = create_schema_viewer()
    standard_tab = create_standard_tables_interface()
    
    # Set up main tabs
    main_tab.children = [query_tab, gen_tab, schema_tab, standard_tab, info_tab]
    main_tab.set_title(0, 'Query & Visualize')
    main_tab.set_title(1, 'Generate Data')
    main_tab.set_title(2, 'Schema Viewer')
    main_tab.set_title(3, 'Standard Tables')
    main_tab.set_title(4, 'Information')
    
    display(main_tab)

def create_query_tab():
    # Query components
    query_input = widgets.Text(
        value='',
        placeholder='Enter your SQL query',
        description='Query:',
        disabled=False,
        layout=widgets.Layout(width='80%')
    )
    
    common_queries = widgets.Dropdown(
        options=[
            ('Show all tables', 'SELECT table_name FROM user_tables'),
            ('Vector embeddings data', 'SELECT * FROM vector_embeddings'),
            ('Table statistics', 'SELECT table_name, num_rows, avg_row_len FROM user_tables'),
            ('Database version', 'SELECT * FROM v$version')
        ],
        description='Quick Queries:',
        layout=widgets.Layout(width='80%')
    )
    
    output = widgets.Output()
    viz_output = widgets.Output()
    
    def query_to_df(query):
        connection = create_connection()
        if connection:
            try:
                return pd.read_sql(query, connection)
            except Exception as e:
                print(f"Error: {str(e)}")
                return None
            finally:
                connection.close()
        return None

    def create_visualizations(df):
        if df is None or df.empty:
            return
        
        with viz_output:
            viz_output.clear_output(wait=True)
            
            # Create basic statistics
            display(widgets.HTML("<h3>Data Statistics</h3>"))
            display(df.describe())
            
            # For numerical columns, create a box plot
            num_cols = df.select_dtypes(include=['int64', 'float64']).columns
            if len(num_cols) > 0:
                fig = px.box(df, y=num_cols)
                fig.update_layout(title="Numerical Columns Distribution")
                fig.show()
            
            # For categorical columns, create a bar plot
            cat_cols = df.select_dtypes(include=['object']).columns
            if len(cat_cols) > 0:
                for col in cat_cols[:3]:
                    value_counts = df[col].value_counts()
                    fig = px.bar(x=value_counts.index, y=value_counts.values,
                               title=f"Distribution of {col}")
                    fig.show()

    def on_query_select(change):
        if change['type'] == 'change' and change['name'] == 'value':
            query_input.value = change['new']

    def on_button_clicked(b):
        with output:
            output.clear_output(wait=True)
            df = query_to_df(query_input.value)
            if df is not None:
                display(widgets.HTML("<h3>Query Results</h3>"))
                display(df)
                create_visualizations(df)

    common_queries.observe(on_query_select, names='value')
    button = widgets.Button(description="Execute Query")
    button.on_click(on_button_clicked)
    
    return widgets.VBox([
        widgets.HTML("<h2>Database Query Interface</h2>"),
        common_queries,
        query_input,
        button,
        output,
        viz_output
    ])

def create_data_generation_tab():
    num_records = widgets.IntSlider(
        value=100,
        min=10,
        max=1000,
        step=10,
        description='Records:',
        layout=widgets.Layout(width='50%')
    )
    
    table_name = widgets.Text(
        value='generated_data',
        placeholder='Enter table name',
        description='Table:',
        layout=widgets.Layout(width='50%')
    )
    
    vector_dim = widgets.IntSlider(
        value=128,
        min=32,
        max=512,
        step=32,
        description='Vector Dim:',
        layout=widgets.Layout(width='50%')
    )
    
    gen_output = widgets.Output()
    
    def generate_sample_data(b):
        with gen_output:
            gen_output.clear_output()
            
            config = {
                'num_records': num_records.value,
                'fields': {
                    'title': {
                        'type': 'string',
                        'choices': ['Research Paper', 'Article', 'Blog Post', 'News'],
                        'null_prob': 0.0
                    },
                    'content_length': {
                        'type': 'int',
                        'min': 100,
                        'max': 5000,
                        'null_prob': 0.0
                    },
                    'relevance_score': {
                        'type': 'float',
                        'min': 0.0,
                        'max': 1.0,
                        'null_prob': 0.1
                    },
                    'embedding': {
                        'type': 'vector',
                        'vector_dim': vector_dim.value,
                        'null_prob': 0.0
                    },
                    'publish_date': {
                        'type': 'date',
                        'min': '2025-01-01',
                        'max': '2025-12-31',
                        'null_prob': 0.0
                    }
                }
            }
            
            df = generate_dummy_data(config)
            print(f"Generated {len(df)} records")
            create_table_from_data(df, table_name.value, vector_cols=['embedding'])
            
            # Display sample and visualization
            display(df.head())
            
            fig = px.scatter(df,
                           x='publish_date',
                           y='content_length',
                           color='relevance_score',
                           title='Generated Data Overview')
            fig.show()
            
            # Show schema after creation
            schema = get_table_schema(table_name.value)
            if schema:
                display(widgets.HTML("<h3>Table Schema:</h3>"))
                display(schema['columns'])
    
    generate_button = widgets.Button(description="Generate Data")
    generate_button.on_click(generate_sample_data)
    
    return widgets.VBox([
        widgets.HTML("<h2>Generate Sample Data</h2>"),
        widgets.HBox([num_records, table_name]),
        vector_dim,
        generate_button,
        gen_output
    ])

def create_info_tab():
    return widgets.VBox([
        widgets.HTML("""
        <h2>Information</h2>
        <h3>Available Tables:</h3>
        <ul>
            <li>vector_embeddings - Stores vector data and embeddings</li>
            <li>user_tables - System table with metadata about all tables</li>
            <li>v$version - Database version information</li>
        </ul>
        <h3>Tips:</h3>
        <ul>
            <li>Use the Quick Queries dropdown for common operations</li>
            <li>Results will be displayed as interactive dataframes</li>
            <li>Automatic visualizations are generated for numerical and categorical data</li>
            <li>Use the Generate Sample Data tab to create test data</li>
            <li>View table schemas in the Schema Viewer tab</li>
            <li>Create standard tables from the Standard Tables tab</li>
        </ul>
        """)
    ])

In [18]:
create_interface()